In [ ]:
#!pip install -e ..

In [ ]:
import os
import gdown
import shutil

from astrohack.gdown_utils import gdown_data
from astrohack.gdown_utils import build_folder_structure

In [ ]:
# Build folder structure & download 

base_name = 'ea25_cal_small_'

datafolder = 'data'
resultsfolder = 'results'

build_folder_structure(datafolder, resultsfolder)

gdown_data(ms_name='ea25_cal_small_before_fixed.split.ms', download_folder=datafolder)
gdown_data(ms_name='ea25_cal_small_after_fixed.split.ms', download_folder=datafolder)

In [ ]:
from astrohack.astrohack_client import astrohack_local_client

log_parms = {'log_level':'DEBUG'}

client = astrohack_local_client(cores=2, memory_limit='8GB', log_parms=log_parms)

In [ ]:
def test_holog_obs_dictionary(holog_obs_dict):
    import os
    import json
    import copy
    
    import numpy as np

    from astrohack._utils._tools import _jsonify

    ref_holog_obj = {}
    ref_holog_obj = copy.deepcopy(holog_obs_dict)

    _jsonify(ref_holog_obj)

    with open("holog_obs_dict.json") as json_file:
        holog_obj = json.load(json_file)
                          
    assert holog_obj == ref_holog_obj, "Error: holog_obs_descrition dictionary has changes unexpectedly."
    
    os.remove("holog_obs_dict.json")    
    

In [ ]:
from astrohack.extract_holog import extract_holog
import numpy as np

beforems = datafolder+'/'+ base_name + 'before_fixed.split.ms'
beforeholog = resultsfolder+'/before.split.holog.zarr'
afterms =  datafolder+'/'+ base_name + 'after_fixed.split.ms'
afterholog = resultsfolder+'/after.split.holog.zarr'

holog_obs_dict = {
    'ddi_0': {
        'map_0': {
            'ant': {
                'ea06': np.array(['ea04']),
                'ea25': np.array(['ea04'])
            },
            'scans': np.array([
                8, 9, 10, 12, 13, 14, 16, 17, 18, 23, 24, 
                25, 27, 28, 29, 31, 32, 33, 38, 39, 40, 
                42, 43, 44, 46, 47, 48, 53, 54, 55, 57
            ])
        }
    }
}


extract_holog(
    ms_name=beforems, 
    holog_name=beforeholog, 
    ddi_sel=[0],
    data_col='CORRECTED_DATA',
    parallel=False,
    write_holog_obs_dict=True,
    overwrite=True,
    reuse_point_zarr=False
)

test_holog_obs_dictionary(holog_obs_dict)

holog_obs_dict = {
    'ddi_0': {
        'map_0': {
            'ant': {
                'ea06': np.array(['ea04']),
                'ea25': np.array(['ea04'])
            },
            'scans': np.array([
                8,  9, 10, 12, 13, 14, 16, 17, 18, 
                23, 24, 25, 27, 28, 29, 31, 32, 33, 
                38, 39, 40, 42, 43, 44, 46, 47, 48, 
                53, 54, 55, 57
            ])
        }
    },
    'ddi_1': {
        'map_0': {
            'ant': {
                'ea06': np.array(['ea04']),
                'ea25': np.array(['ea04'])
            },
            'scans': np.array([
                8,  9, 10, 12, 13, 14, 16, 17, 18, 
                23, 24, 25, 27, 28, 29, 31, 32, 33, 
                38, 39, 40, 42, 43, 44, 46, 47, 48, 
                53, 54, 55, 57
            ])
        }
    }
}


print('***'*20)
holog_mds_after, _ = extract_holog(
    ms_name=afterms, 
    holog_name=afterholog,
    data_col='CORRECTED_DATA',
    parallel=False,
    write_holog_obs_dict=True,
    overwrite=True,
    reuse_point_zarr=False
)

test_holog_obs_dictionary(holog_obs_dict)


In [ ]:
holog_mds_after

In [ ]:
import numpy as np
from astrohack import holog

#cell_size = np.array([-0.0006442, 0.0006442])
#grid_size = np.array([31, 31])
grid_interpolation_mode = 'linear' #'nearest' 'linear' 'cubic'
chan_average = True
scan_average = True

holog(holog_name=beforeholog, padding_factor=50, 
      grid_interpolation_mode=grid_interpolation_mode,
      chan_average = chan_average,
      scan_average = scan_average,
      overwrite=True,
      phase_fit=True,
      apply_mask=True,
      to_stokes=True,
      parallel=True
     )

print('***'*20)
holog(holog_name=afterholog, padding_factor=50, 
      grid_interpolation_mode=grid_interpolation_mode,
      chan_average = chan_average,
      scan_average = scan_average,
      overwrite=True,
      phase_fit=True,
      apply_mask=True,
      to_stokes=True,
      parallel=True
     )

In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import scipy
import matplotlib.patches as patches

from scipy import constants

beforeimage = resultsfolder+'/before.split.image.zarr'
afterimage = resultsfolder+'/after.split.image.zarr'

plt.close('all')

chan = 0

ds = xr.open_zarr(afterimage+'/ant_ea25/ddi_0')
wavelength = scipy.constants.speed_of_light/ds.chan.values[chan]
l = ds.l.values
m = ds.m.values
u = ds.u.values*wavelength
v = ds.v.values*wavelength

print(u.min(),u.max())

plt.figure()
plt.imshow(np.abs(ds.BEAM[0,chan,0,:,:]),extent=[l.min(), l.max(), m.min(), m.max()])
plt.colorbar()
plt.show()

circle = patches.Circle((0,0), 12.5, fill=False, color='white', alpha=0.7, linewidth=2)

fig, ax =plt.subplots()
plt.imshow(np.abs(ds.APERTURE[0,chan,0,:,:]),extent=[u.min(), u.max(), v.min(), v.max()])
#ax.add_patch(circle)
plt.show()

ds

In [ ]:
from astrohack.panel import panel

panel_model = 'rigid'

before_panel = panel(
    image_name=beforeimage, 
    panel_model=panel_model,
    parallel=True,
    overwrite=True
)
after_panel = panel(
    image_name=afterimage, 
    panel_model=panel_model,
    parallel=True,
    overwrite=True
)

In [ ]:
beforescrews = resultsfolder+'/before-screws.txt'
afterscrews = resultsfolder+'/after-screws.txt'
before_ant = before_panel.get_antenna('ant_ea25', 'ddi_0')
after_ant = after_panel.get_antenna('ant_ea25', 'ddi_0')

before_ant.export_screws(beforescrews, unit="mils")
after_ant.export_screws(afterscrews, unit="mils")

In [ ]:
def relative_difference(mean, expected):  
    return 100.*2*abs(mean - expected)/(abs(mean) + abs(expected))

# Replace whole function; get panel_mds
def extract_panel_data(fname, panel_list):
    data = open(fname, 'r')
    panels = []
    for line in data:
        wrds = line.split('  ')
        if wrds[0] in panel_list:
            nmbrs = line.split()
            panel = [0, 0, 0, 0]
            for i in range(4):
                panel[-1-i] = float(nmbrs[-1-i])
            panels.append(panel)
            
    return np.array(panels)
    
def compare_before_after(panel_list, expected_shift): 
    # Replace with mds query
    after = extract_panel_data(afterscrews, panel_list)
    before = extract_panel_data(beforescrews, panel_list)
    
    diffs = after - before
    
    mean_shift = np.mean(diffs, axis=1)
    
    percent_shift = relative_difference(mean_shift, expected_shift)
    
    return mean_shift, percent_shift

def test_improvement(pname, diff, unit):
    abso = np.abs(diff)
    if abso > 1e-7:
        if diff < 0:
            print('Panel {0:s} has improved by {1:.20f} {2:s}'.format(pname, abso, unit))
        else:
            print('Panel {0:s} is worse by {1:.20f} {2:s}'.format(pname, abso, unit))
    else:
        print('Panel {0:s} is unchanged'.format(pname))

def is_it_an_improvement(mean_shift, percent_shift, ref_mean_shift):
    sumcomp = np.mean(ref_mean_shift - mean_shift)
    print(ref_mean_shift - mean_shift)
        
    if abs(sumcomp) < 1e-7:
        print('No changes!')
    else:
        print('Results have changed')
        for i in range(4):
            pname = panel_list[i]
            if expected_shift[i] == 0:
                diff = np.abs(mean_shift[i]-ref_mean_shift[i])
                test_improvement(pname, diff, 'mils')
            else:
                diff = np.abs(percent_shift[i]) - np.abs(ref_percent_shift[i])
                test_improvement(pname, diff, '%')


panel_list = ['3-4', '5-27', '5-37', '5-38']
expected_shift  = np.array([-100, 75, 0, 150])

ref_mean_shift = np.array([-112.2325,   73.225,    -1.455,   139.04  ])
ref_percent_shift = relative_difference(ref_mean_shift, expected_shift)
                
                
mean_shift, percent_shift = compare_before_after(panel_list, expected_shift)
is_it_an_improvement(mean_shift, percent_shift, ref_mean_shift)

In [ ]:
from astrohack.dio import open_panel

def verify_tests(
    panel_list=['3-4', '5-27', '5-37', '5-38'], 
    expected_shift=[np.array([-100, 75, 0, 150])],
    ref_mean_shift = np.array([-112.2325, 73.225, -1.455, 139.04  ]),
    antenna='ant_ea25',
    ddi='ddi_0'
):
    
    M_TO_MILS = 39370.1
    
    before_mds = open_panel('results/before.split.panel.zarr')
    after_mds = open_panel('results/after.split.panel.zarr')
    
    before_shift = before_mds[antenna][ddi].sel(labels=panel_list).PANEL_SCREWS.values*M_TO_MILS
    after_shift = after_mds[antenna][ddi].sel(labels=panel_list).PANEL_SCREWS.values*M_TO_MILS
    
    difference = after_shift - before_shift
    
    mean_shift = np.mean(difference, axis=1)
    
    delta_mean_shift = np.abs(mean_shift - expected_shift)
    delta_ref_shift = np.abs(ref_mean_shift - expected_shift)
        
    delta_shift = delta_mean_shift - delta_ref_shift  # New corrections - old corrections --> delta if delta < 0 ==> we improved.
    
    if np.any(np.abs(delta_shift) > 0.0039): # This is approximately a micron shift
        print("There were changes!")
        for i, delta in enumerate(delta_shift[0]):
            if delta < 0:
                print("{panel}, improved by {delta} mils".format(panel=panel_list[i], delta=delta))
            else:
                print("{panel}, got worse by {delta} mils".format(panel=panel_list[i], delta=delta))
    

In [ ]:
verify_tests()